In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==3.1.0

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
classifier = pipeline("zero-shot-classification", device=0)

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def evaluation_results(true_label, pred_label,labels):
  acc = accuracy_score(true_label,pred_label)
  print("Accuracy:",acc)
  print(classification_report(true_label,pred_label))
  cm = pd.DataFrame(confusion_matrix(true_label,pred_label), columns=labels, 
                  index=labels)
  print(cm)
  #print(pred_label.value_counts())

# **ZSL for 2 way classification**

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/NLP_Dissertation/Datasets/nli_pet.csv')
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   compound          670 non-null    object 
 1   sentence1         670 non-null    object 
 2   sentence2         670 non-null    object 
 3   compositionality  670 non-null    float64
 4   gold_label        670 non-null    object 
dtypes: float64(1), object(4)
memory usage: 26.3+ KB


In [ ]:
sequences = list(ds['sentence1'])

##**Default template**

In [ ]:
ds['label'] = np.where(ds['compositionality'] >= 2.5, 'literal', 'idiomatic')
ds['label'].value_counts(normalize=True)

literal      0.50597
idiomatic    0.49403
Name: label, dtype: float64

In [ ]:
ds['label'].value_counts()

literal      339
idiomatic    331
Name: label, dtype: int64

In [ ]:
labels=['literal', 'idiomatic']

In [ ]:
predictions = classifier(sequences, labels)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])  

In [ ]:
from collections import Counter
Counter(pred)

Counter({'idiomatic': 150, 'literal': 520})

In [ ]:
#ds['pred_label'] = pred
evaluation_results(ds['label'], pred,['idiomatic','literal'])

Accuracy: 0.48208955223880595
              precision    recall  f1-score   support

   idiomatic       0.45      0.20      0.28       331
     literal       0.49      0.76      0.60       339

    accuracy                           0.48       670
   macro avg       0.47      0.48      0.44       670
weighted avg       0.47      0.48      0.44       670

           idiomatic  literal
idiomatic         67      264
literal           83      256


## **Template-1**

In [ ]:
ds['label'] = np.where(ds['compositionality'] >= 2.5, 'literal', 'non-literal')
ds['label'].value_counts(normalize=True)

literal        0.50597
non-literal    0.49403
Name: label, dtype: float64

In [ ]:
nli_template = "This sentence is {}."
labels = ['literal', 'non-literal']

In [ ]:
predictions = classifier(sequences, labels, hypothesis_template=nli_template)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])

Counter(pred)

Counter({'literal': 2, 'non-literal': 668})

In [ ]:
evaluation_results(ds['label'], pred, labels)

Accuracy: 0.49402985074626865
              precision    recall  f1-score   support

     literal       0.50      0.00      0.01       339
 non-literal       0.49      1.00      0.66       331

    accuracy                           0.49       670
   macro avg       0.50      0.50      0.33       670
weighted avg       0.50      0.49      0.33       670

             literal  non-literal
literal            1          338
non-literal        1          330


## **Template-2**

In [ ]:
ds['label'] = np.where(ds['compositionality'] >= 2.5, 'True', 'False')
ds['label'].value_counts(normalize=True)

True     0.50597
False    0.49403
Name: label, dtype: float64

In [ ]:
nli_template = "{}, this sentence is expressed literally."
labels = ['True', 'False']

In [ ]:
predictions = classifier(sequences, labels, hypothesis_template=nli_template)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])

Counter(pred)  

Counter({'False': 46, 'True': 624})

In [ ]:
evaluation_results(ds['label'], pred, ['False', 'True'])

Accuracy: 0.5477611940298508
              precision    recall  f1-score   support

       False       0.80      0.11      0.20       331
        True       0.53      0.97      0.69       339

    accuracy                           0.55       670
   macro avg       0.67      0.54      0.44       670
weighted avg       0.66      0.55      0.44       670

       False  True
False     37   294
True       9   330


# **ZSL for multiclass classification**

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/NLP_Dissertation/Datasets/nli_pet.csv')
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   compound          670 non-null    object 
 1   sentence1         670 non-null    object 
 2   sentence2         670 non-null    object 
 3   compositionality  670 non-null    float64
 4   gold_label        670 non-null    object 
dtypes: float64(1), object(4)
memory usage: 26.3+ KB


In [ ]:
ds['gold_label'].value_counts(normalize=True)

entailment       0.428358
contradiction    0.405970
neutral          0.165672
Name: gold_label, dtype: float64

In [ ]:
ds['gold_label'].value_counts()

entailment       287
contradiction    272
neutral          111
Name: gold_label, dtype: int64

In [ ]:
sequences = list(ds['sentence1'])

##**Template 1**

In [ ]:
#ds['label'] = np.where(ds['gold_label'] == 'entailment', 'literal', np.where(ds['gold_label'] == 'contradiction', 'idiomatic', 'neutral'))
#ds['label'].value_counts(normalize=True)

In [ ]:
#labels=['literal', 'idiomatic']

In [ ]:
ds['label'] = np.where(ds['gold_label'] == 'entailment', 'Right', np.where(ds['gold_label'] == 'contradiction', 'Wrong', 'Maybe'))
ds['label'].value_counts(normalize=True)

Right    0.428358
Wrong    0.405970
Maybe    0.165672
Name: label, dtype: float64

In [ ]:
nli_template = "{}, this sentence is expressed literally."
labels = ['Right', 'Wrong', 'Maybe']

In [ ]:
predictions = classifier(sequences, labels)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])

Counter(pred)  

Counter({'Maybe': 374, 'Right': 187, 'Wrong': 109})

In [ ]:
evaluation_results(ds['label'], pred, ['Maybe', 'Right', 'Wrong'])

Accuracy: 0.30895522388059704
              precision    recall  f1-score   support

       Maybe       0.16      0.54      0.25       111
       Right       0.45      0.29      0.35       287
       Wrong       0.58      0.23      0.33       272

    accuracy                           0.31       670
   macro avg       0.40      0.35      0.31       670
weighted avg       0.45      0.31      0.33       670

       Maybe  Right  Wrong
Maybe     60     29     22
Right    179     84     24
Wrong    135     74     63


## **Template-2**

In [ ]:
ds['label'] = np.where(ds['gold_label'] == 'entailment', 'Yes', np.where(ds['gold_label'] == 'contradiction', 'No', 'Maybe'))
ds['label'].value_counts(normalize=True)

Yes      0.428358
No       0.405970
Maybe    0.165672
Name: label, dtype: float64

In [ ]:
nli_template = "{}, this sentence is expressed literally."
labels = ['Yes', 'No', 'Maybe']

In [ ]:
predictions = classifier(sequences, labels, hypothesis_template=nli_template)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])

Counter(pred)  

Counter({'Maybe': 282, 'No': 5, 'Yes': 383})

In [ ]:
evaluation_results(ds['label'], pred, ['Maybe', 'No', 'Yes'])

Accuracy: 0.31343283582089554
              precision    recall  f1-score   support

       Maybe       0.18      0.45      0.25       111
          No       0.80      0.01      0.03       272
         Yes       0.41      0.54      0.47       287

    accuracy                           0.31       670
   macro avg       0.46      0.34      0.25       670
weighted avg       0.53      0.31      0.25       670

       Maybe  No  Yes
Maybe     50   1   60
No       101   4  167
Yes      131   0  156


## **Template-3**

In [ ]:
ds['label'] = np.where(ds['gold_label'] == 'entailment', 'True', np.where(ds['gold_label'] == 'contradiction', 'False', 'Maybe'))
ds['label'].value_counts(normalize=True)

True     0.428358
False    0.405970
Maybe    0.165672
Name: label, dtype: float64

In [ ]:
nli_template = "{}, this sentence is expressed literally."
labels = ['True', 'False', 'Maybe']

In [ ]:
predictions = classifier(sequences, labels, hypothesis_template=nli_template)

In [ ]:
#predictions

In [ ]:
pred = []
for x in range(0,len(predictions)):
  pred.append(predictions[x]['labels'][0])

Counter(pred)  

Counter({'False': 16, 'Maybe': 476, 'True': 178})

In [ ]:
evaluation_results(ds['label'], pred, ['False', 'Maybe', 'True'])

Accuracy: 0.26417910447761195
              precision    recall  f1-score   support

       False       0.62      0.04      0.07       272
       Maybe       0.18      0.76      0.29       111
        True       0.47      0.29      0.36       287

    accuracy                           0.26       670
   macro avg       0.42      0.36      0.24       670
weighted avg       0.48      0.26      0.23       670

       False  Maybe  True
False     10    189    73
Maybe      5     84    22
True       1    203    83


# References

1. https://joeddav.github.io/blog/2020/05/29/ZSL.html

2. https://huggingface.co/facebook/bart-large-mnli

3. Yin, W., Hay, J., and Roth, D.Benchmarking zero-shot text classification: Datasets,evaluation and entailment approach, 2019.